<center><h1><strong>Diseño de una Montaña Rusa</strong></h1></center>  
<center><img src="Montaña Rusa.jpg" width=450 height=270></center>  

### **Sección: 305C1**
### **Estudiantes:**
Manuel Nava 30.822.007  
Juan Wu 30.391.117  
Enrique Gonzalez 31.456.812